In [1]:
import sys
sys.path.append('/home/lessa/smodels')
from smodels.experiment.databaseObj import Database
from smodels.tools.physicsUnits import GeV,fb,pb
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from smodels.tools.databaseBrowser import Browser
import numpy as np

/home/lessa/.local/lib/python2.7/site-packages/scipy/special/__init__.py:640: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ufuncs import *
/home/lessa/.local/lib/python2.7/site-packages/scipy/linalg/basic.py:17: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._solve_toeplitz import levinson
/home/lessa/.local/lib/python2.7/site-packages/scipy/linalg/__init__.py:207: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._decomp_update import *
/home/lessa/.local/lib/python2.7/site-packages/scipy/special/_ellip_harm.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._ellip_harm_2 import _ellipsoid, _ellipsoid_norm
/home/lessa/.local/lib/python2.7/site-packages/scipy/interpolate/_bsplines.py:10: RuntimeWarning: numpy.dtype size changed, may indicate binary

In [2]:
dbname="http://smodels.hephy.at/database/unittest200"
db = Database(dbname, discard_zeroes = False)

loading >...................................................done.


In [3]:
exp = db.getExpResults(analysisIDs=['CMS-PAS-SUS-12-026'])[0]

In [4]:
tx = exp.getTxNames()[0]
print tx,tx.constraint

T1tttt [[[t,t]],[[t,t]]]


In [5]:
m = [[1000.*GeV, 222.*GeV]]*2
print(tx.txnameData.getValueFor(m))

4.06E+01 [fb]


In [6]:
self = tx.txnameData
massarray = m
porig = self.removeUnits(massarray)
porig = self.formatInput(porig,self.dataShape) #Remove entries which match inclusives
porig = self.flattenArray(porig) ## flatten        
self.massarray = massarray ## only for bookkeeping and better error msgs

p = ((np.matrix(porig)[0] - self.delta_x )).tolist()[0]
P = np.dot(p,self._V)  ## rotate
print('P=',P,self.interpolate(P[:self.dimensionality]))

('P=', array([ 9.66151423e+00,  1.64050820e+02, -2.16529791e-14,  2.71043763e-14]), 40.62376)


In [7]:
self = tx.txnameData
massarray = m
porig = self.removeUnits(massarray)
porig = self.formatInput(porig,self.dataShape) #Remove entries which match inclusives
porig = self.flattenArray(porig) ## flatten        
self.massarray = massarray ## only for bookkeeping and better error msgs


print(np.matrix(porig)[0])
print(np.array([porig]))

p = ((np.matrix(porig)[0] - self.delta_x )).tolist()[0]
print('p=',p)
p = ((np.array([porig]) - self.delta_x )).tolist()[0]
print('p=',p)
P = np.dot(p,self._V)  ## rotate
print('P=',P,self.interpolate(P[:self.dimensionality]))

[[1000.  222. 1000.  222.]]
[[1000.  222. 1000.  222.]]
('p=', [104.5454545454545, -50.72727272727275, 104.5454545454545, -50.72727272727275])
('p=', [104.5454545454545, -50.72727272727275, 104.5454545454545, -50.72727272727275])
('P=', array([ 9.66151423e+00,  1.64050820e+02, -2.16529791e-14,  2.71043763e-14]), 40.62376)


In [8]:
uvw = [pt[:3]]
fill_value=np.nan
tol = sys.float_info.epsilon * 1e06
simplex = tx.txnameData.tri.find_simplex(uvw, tol=tol)
vertices = np.take(tx.txnameData.tri.simplices, simplex, axis=0)
temp = np.take(tx.txnameData.tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = uvw - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
tx.txnameData.vtx = vertices
tx.txnameData.wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
if type(tx.txnameData.xsec[0]) == float:
    values = np.array ( [ float(x) for x in tx.txnameData.xsec ] )
else:
    values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
print 'v=',tx.txnameData.wts
ret = np.einsum('nj,nj->n', np.take(values, tx.txnameData.vtx), tx.txnameData.wts)
print 'r=',ret
with np.errstate(invalid='ignore'):
    ret[np.any(tx.txnameData.wts < -1e-10, axis=1)] = fill_value
print ret

NameError: name 'pt' is not defined

In [ ]:
fig.clear()
plt.triplot(xpts,ypts)
plt.plot([ptm[0]], [ptm[1]],'ro',xpts,ypts,'bo')
plt.show()

In [ ]:
tri = tx.txnameData.tri
simplex = tri.find_simplex([pt])
vertices = np.take(tri.simplices, simplex, axis=0)
temp = np.take(tri.transform, simplex, axis=0)
d=temp.shape[2]
delta = [pt] - temp[:, d]
bary = np.einsum('njk,nk->nj', temp[:, :d, :], delta)
vtx = vertices
wts = np.hstack((bary, 1 - bary.sum(axis=1, keepdims=True)))
v=tx.txnameData.xsec
values = np.array ( [ x.asNumber() for x in tx.txnameData.xsec ] )
ret = np.einsum('nj,nj->n', np.take(values, vtx), wts)
with np.errstate(invalid='ignore'):
    ret[np.any(wts < -1e-10, axis=1)] = np.nan
print ret[0]

In [ ]:
vertices

In [ ]:
pt

In [ ]:
m = [[347.406576078*GeV,166.767198233*GeV],[347.406576078*GeV,166.767198233*GeV]]
porig = tx.txnameData.flattenMassArray(m)
ptA = ( (np.matrix(porig)[0] - tx.txnameData.delta_x ) ).tolist()[0]
ptA = np.dot(ptA,tx.txnameData._V)  ## rotate
print ptA,tx.txnameData.countNonZeros(ptA)

In [ ]:
print tx.txnameData.getValueFor(m)